<a href="https://colab.research.google.com/github/Tonmf/ProyectoIML/blob/master/Casos/Caso_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reporte de MLT de Barcelona Telco

## Caso 5: Fábrica Milwaukee (aka Bosch)


## Data

Source: https://www.kaggle.com/c/bosch-production-line-performance

In [ ]:
import os              # Package to use directory command to list files
import numpy as np     # linear algebra
import pandas as pd    # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns  # Used for plotting the graph
import matplotlib.pyplot as plt
import re

from scipy import stats # for Statistics
from statistics import mean
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from matplotlib import rcParams
%matplotlib inline

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
data = pd.read_csv('/content/gdrive/My Drive/IML/Proyecto Final/train_numeric.csv',nrows=4000)

## Data Exploration

In [ ]:
print(data.shape)
data.head(5)

(4000, 970)


,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S2_F32,L0_S2_F36,L0_S2_F40,L0_S2_F44,L0_S2_F48,L0_S2_F52,L0_S2_F56,L0_S2_F60,L0_S2_F64,L0_S3_F68,L0_S3_F72,L0_S3_F76,L0_S3_F80,L0_S3_F84,L0_S3_F88,L0_S3_F92,L0_S3_F96,L0_S3_F100,L0_S4_F104,L0_S4_F109,L0_S5_F114,L0_S5_F116,L0_S6_F118,L0_S6_F122,L0_S6_F132,...,L3_S45_F4126,L3_S45_F4128,L3_S45_F4130,L3_S45_F4132,L3_S47_F4138,L3_S47_F4143,L3_S47_F4148,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S47_F4168,L3_S47_F4173,L3_S47_F4178,L3_S47_F4183,L3_S47_F4188,L3_S48_F4193,L3_S48_F4196,L3_S48_F4198,L3_S48_F4200,L3_S48_F4202,L3_S48_F4204,L3_S49_F4206,L3_S49_F4211,L3_S49_F4216,L3_S49_F4221,L3_S49_F4226,L3_S49_F4231,L3_S49_F4236,L3_S50_F4241,L3_S50_F4243,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,0.083,-0.273,-0.273,-0.271,0.167,-0.213,-0.023,-0.192,-0.088,0.001,0.0,0.01,-0.223,-0.030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001,-0.004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,-0.147,0.250,0.250,0.057,-0.079,-0.013,0.011,0.008,-0.060,-0.005,0.0,0.01,-0.223,-0.077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.073,0.138,-0.336,0.506,-0.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,-0.046,-0.253,-0.253,0.147,-0.007,-0.013,0.120,0.008,-0.231,0.005,0.0,0.01,0.050,0.056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.038,-0.001,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,-0.099,0.042,0.042,-0.012,-0.046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009,-0.272,-0.051,0.037,0.004,0.0,-0.081,0.311,0.003,0.021,0.015,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
data.describe()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S2_F32,L0_S2_F36,L0_S2_F40,L0_S2_F44,L0_S2_F48,L0_S2_F52,L0_S2_F56,L0_S2_F60,L0_S2_F64,L0_S3_F68,L0_S3_F72,L0_S3_F76,L0_S3_F80,L0_S3_F84,L0_S3_F88,L0_S3_F92,L0_S3_F96,L0_S3_F100,L0_S4_F104,L0_S4_F109,L0_S5_F114,L0_S5_F116,L0_S6_F118,L0_S6_F122,L0_S6_F132,...,L3_S45_F4126,L3_S45_F4128,L3_S45_F4130,L3_S45_F4132,L3_S47_F4138,L3_S47_F4143,L3_S47_F4148,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S47_F4168,L3_S47_F4173,L3_S47_F4178,L3_S47_F4183,L3_S47_F4188,L3_S48_F4193,L3_S48_F4196,L3_S48_F4198,L3_S48_F4200,L3_S48_F4202,L3_S48_F4204,L3_S49_F4206,L3_S49_F4211,L3_S49_F4216,L3_S49_F4221,L3_S49_F4226,L3_S49_F4231,L3_S49_F4236,L3_S50_F4241,L3_S50_F4243,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
count,4000.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2327.000000,2327.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.0,1166.000000,1166.000000,1166.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.0,1160.000000,1160.000000,1160.000000,1139.000000,1139.000000,1189.000000,1189.000000,1160.000000,1160.000000,1160.000000,...,194.0,194.0,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.0,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.0,194.000000,194.000000,92.000000,92.000000,92.000000,92.000000,92.0,92.0,92.000000,102.000000,102.000000,102.000000,102.000000,102.0,102.000000,102.000000,194.000000,194.000000,194.000000,194.000000,4000.00000
mean,4018.952500,-0.001055,-0.001595,0.004938,0.004402,0.001665,0.007874,0.000389,0.002804,-0.001648,-0.002217,0.001983,0.001995,0.002849,0.002853,-0.004338,0.015549,-0.004350,-0.000332,-0.000110,0.0,-0.001664,0.000624,0.006369,0.000466,0.001101,0.002017,0.002544,0.000182,0.0,0.001672,-0.008123,-0.001122,0.001487,0.000307,-0.000683,-0.007216,-0.016172,0.021811,-0.000716,...,0.0,0.0,0.000010,0.000077,0.005753,-0.000124,0.006572,-0.001381,0.001041,0.019814,0.006572,0.0,0.000222,0.000082,0.000072,-0.023242,0.008294,-0.000134,0.0,0.000149,0.000093,-0.001011,0.001978,-0.000011,0.000304,0.0,0.0,0.022739,-0.025118,-0.004627,-0.000020,0.000010,0.0,0.000186,-0.009804,-0.000005,0.000005,0.000155,0.000196,0.00575
std,2307.764835,0.078488,0.091779,0.213230,0.212953,0.094116,0.160903,0.019176,0.103750,0.113865,0.110104,0.201989,0.201972,0.103925,0.108682,0.050767,0.209693,0.068348,0.264412,0.006543,0.0,0.056821,0.262852,0.061439,0.046309,0.250478,0.054078,0.271901,0.006336,0.0,0.052887,0.242717,0.053020,0.054708,0.014361,0.127981,0.150973,0.248974,0.345922,0.106862,...,0.0,0.0,0.000101,0.000938,0.096644,0.003160,0.103856,0.084464,0.176276,0.102905,0.103856,0.0,0.001904,0.000570,0.001005,0.404670,0.131445,0.061192,0.0,0.001040,0.001044,0.000104,0.051298,0.000104,0.002237,0.0,0.0,0.250418,0.118425,0.150980,0.000139,0.000099,0.0,0.001031,0.251041,0.000072,0.000072,0.001549,0.001686,0.07562
min,4.000000,-0.291000,-0.362000,-0.397000,-0.397000,-0.404000,-0.566000,-0.044000,-0.232000,-0.393000,-0.334000,-0.340000,-0.340000,-0.287000,-0.253000,-0.213000,-0.208000,-0.192000,-0.273000,-0.037000,0.0,-0.190000,-0.266000,-0.084000,-0.191000,-0.274000,-0.051000,-0.350000,-0.036000,0.0,-0.181000,-0.290000,-0.088000,-0.142000,-0.046000,-0.342000,-0.207000,-0.336000,-0.323000,-0.142000,...,0.0,0.0,0.000000,0.000000,-0.217000,-0.010000,-0.224000,-0.239000,-0.418000,-0.287000,-0.224000,0.0,0.000000,0.000000,0.000000,-0.520000,-0.327000,-0.573000,0.0,0.000000,0.000000,-0.002000,-0.075000,-0.001000,0.000000,0.0,0.0,-0.249000,-0.531000,-0.257000,-0.001000,0.000000,0.0,0.000000,-0.250000,-0.001000,0.000000,0.000000,0.000000,0.00000
25%,2058.000000,-0.055000,-0.056000,-0.179000,-0.179000,-0.056000,-0.0

In [ ]:
data.columns

Index(['Id', 'L0_S0_F0', 'L0_S0_F2', 'L0_S0_F4', 'L0_S0_F6', 'L0_S0_F8',
       'L0_S0_F10', 'L0_S0_F12', 'L0_S0_F14', 'L0_S0_F16',
       ...
       'L3_S50_F4245', 'L3_S50_F4247', 'L3_S50_F4249', 'L3_S50_F4251',
       'L3_S50_F4253', 'L3_S51_F4256', 'L3_S51_F4258', 'L3_S51_F4260',
       'L3_S51_F4262', 'Response'],
      dtype='object', length=970)

In [ ]:
data.dtypes

Id                int64
L0_S0_F0        float64
L0_S0_F2        float64
L0_S0_F4        float64
L0_S0_F6        float64
                 ...   
L3_S51_F4256    float64
L3_S51_F4258    float64
L3_S51_F4260    float64
L3_S51_F4262    float64
Response          int64
Length: 970, dtype: object

In [ ]:
totalt = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([totalt, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
L1_S25_F2181,3997,0.99925
L1_S25_F2184,3997,0.99925
L1_S25_F2187,3997,0.99925
L1_S25_F2190,3997,0.99925
L1_S25_F2193,3997,0.99925
...,...,...
L3_S37_F3946,204,0.05100
L3_S37_F3948,204,0.05100
L3_S37_F3950,204,0.05100
Response,0,0.00000


In [ ]:
796/970 *100

82.06185567010309

In [ ]:
df_agrup = data.groupby(by=['Response'])
df_agrup.count()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S2_F32,L0_S2_F36,L0_S2_F40,L0_S2_F44,L0_S2_F48,L0_S2_F52,L0_S2_F56,L0_S2_F60,L0_S2_F64,L0_S3_F68,L0_S3_F72,L0_S3_F76,L0_S3_F80,L0_S3_F84,L0_S3_F88,L0_S3_F92,L0_S3_F96,L0_S3_F100,L0_S4_F104,L0_S4_F109,L0_S5_F114,L0_S5_F116,L0_S6_F118,L0_S6_F122,L0_S6_F132,...,L3_S45_F4124,L3_S45_F4126,L3_S45_F4128,L3_S45_F4130,L3_S45_F4132,L3_S47_F4138,L3_S47_F4143,L3_S47_F4148,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S47_F4168,L3_S47_F4173,L3_S47_F4178,L3_S47_F4183,L3_S47_F4188,L3_S48_F4193,L3_S48_F4196,L3_S48_F4198,L3_S48_F4200,L3_S48_F4202,L3_S48_F4204,L3_S49_F4206,L3_S49_F4211,L3_S49_F4216,L3_S49_F4221,L3_S49_F4226,L3_S49_F4231,L3_S49_F4236,L3_S50_F4241,L3_S50_F4243,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262
Response,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3977,2316,2316,2316,2316,2316,2316,2316,2316,2316,2316,2316,2316,2318,2318,1164,1164,1164,1164,1164,1164,1164,1164,1164,1153,1153,1153,1153,1153,1153,1153,1153,1153,1133,1133,1186,1186,1154,1154,1154,...,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,91,91,91,91,91,91,91,102,102,102,102,102,102,102,193,193,193,193
1,23,9,9,9,9,9,9,9,9,9,9,9,9,9,9,2,2,2,2,2,2,2,2,2,7,7,7,7,7,7,7,7,7,6,6,3,3,6,6,6,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1


De este recuento podemos sacar un par de cosas interesantes: 

1.   Sólo hay 23 casos en que tenemos una pieza mala
2.   De un vistazo rápido vemos que en cierta líneas hay más 1's que en otras.

In [ ]:
23/4000 *100

0.575

## Algoritmo de Machine Learning

## Resultados

 ## Discusión y Conclusiones

## Linias de desarrollo futuro

### Annexo 1


### Annexo 2